Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
Если товар не встречался в обучающей выборке, его популярность равна 0.
Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.
Задание

1. На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать)

2. Реализуйте два алгоритма рекомендаций:
сортировка просмотренных id по популярности (частота появления в просмотренных),
сортировка просмотренных id по покупаемости (частота появления в покупках).

3. Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

Дополнительные вопросы

Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.
Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [43]:
import pandas as pd
from itertools import chain
from collections import Counter, OrderedDict
import numpy as np

Read the file and on it's base create the list "arr", which contain the number of dictionaries. Each dictionary coresponds one session and contains the lists "viewed" and "purchased" with corresponding ids.

In [44]:
f = open("coursera_sessions_train.txt","r")
arr = []
i = 0
for line in f.readlines():
    line = line.split(";")
    line[0] = list(map(int, line[0].split(",")))
    if line[1] != '\n':
      line[1] = line[1][:len(line[1])-1]
      line[1] = list(map(int, line[1].split(",")))
    else:
      line[1] = []
    dict1 = {}
    dict1["viewed"] = line[0]
    dict1["purchased"] = line[1]
    arr.append(dict1)

Create Counters c1 and c2, which contain information about the quantity of viewed and purchased ids respectively.

In [45]:
c1 = Counter()
c2 = Counter()
for el in arr:
  c1.update(el["viewed"])
  c2.update(el["purchased"])
print("c1:\n" , c1)
print("c2:\n" , c2)

c1:
 Counter({73: 677, 158: 641, 204: 396, 262: 387, 162: 318, 7: 312, 137: 306, 1185: 284, 6: 283, 170: 280, 800: 253, 5202: 227, 8: 225, 609: 213, 3149: 213, 3324: 204, 1346: 199, 751: 197, 1184: 186, 1933: 186, 1283: 186, 1844: 186, 325: 184, 551: 177, 4604: 177, 1334: 174, 42149: 174, 259: 173, 1852: 172, 70238: 170, 72: 167, 258: 167, 758: 166, 85: 165, 1342: 165, 343: 164, 2290: 163, 5501: 160, 1323: 159, 3718: 158, 301: 156, 115: 154, 879: 153, 2922: 153, 3697: 153, 6974: 151, 260: 150, 1214: 149, 3286: 149, 2084: 149, 28: 148, 363: 147, 255: 147, 302: 144, 71: 141, 791: 140, 1349: 140, 1595: 140, 875: 139, 1934: 139, 884: 138, 1814: 138, 2845: 136, 553: 135, 5894: 133, 759: 133, 1651: 131, 3882: 131, 114: 129, 2397: 129, 11027: 129, 1949: 129, 72586: 127, 552: 125, 469: 124, 1011: 122, 966: 122, 2287: 122, 1843: 122, 316: 121, 2965: 121, 6882: 121, 594: 121, 1869: 120, 3287: 120, 2070: 120, 4535: 119, 461: 119, 746: 118, 1140: 118, 1277: 116, 2513: 116, 591: 115, 2294: 115, 937

Create the function, which:

1. delete dublicated elements
2. save the original order
3. sorted ids by their frecuency (corresponding Count)

In [46]:
def id_sort(ids, c):
  ids = list(OrderedDict.fromkeys(ids))
  ids = sorted(ids, key=lambda x: c[x], reverse=True)
  return ids

In [47]:
arr[:10]

[{'purchased': [], 'viewed': [0, 1, 2, 3, 4, 5]},
 {'purchased': [], 'viewed': [9, 10, 11, 9, 11, 12, 9, 11]},
 {'purchased': [], 'viewed': [16, 17, 18, 19, 20, 21]},
 {'purchased': [], 'viewed': [24, 25, 26, 27, 24]},
 {'purchased': [], 'viewed': [34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]},
 {'purchased': [], 'viewed': [42]},
 {'purchased': [], 'viewed': [47, 48, 49]},
 {'purchased': [67, 60, 63],
  'viewed': [59, 60, 61, 62, 60, 63, 64, 65, 66, 61, 67, 68, 67]},
 {'purchased': [], 'viewed': [71, 72, 73, 74]},
 {'purchased': [], 'viewed': [76, 77, 78]}]

In [48]:
arr_v = arr[:]

Apply the id_sort function to each session in arr_v.

In [49]:
for i in range(len(arr_v)):
  arr_v[i]["viewed"] = id_sort(arr_v[i]["viewed"], c1) 

View the result:

In [50]:
arr_v[:10]

[{'purchased': [], 'viewed': [4, 2, 3, 0, 1, 5]},
 {'purchased': [], 'viewed': [12, 9, 10, 11]},
 {'purchased': [], 'viewed': [17, 20, 16, 19, 18, 21]},
 {'purchased': [], 'viewed': [27, 24, 26, 25]},
 {'purchased': [], 'viewed': [35, 34, 36, 37, 38, 39]},
 {'purchased': [], 'viewed': [42]},
 {'purchased': [], 'viewed': [48, 47, 49]},
 {'purchased': [67, 60, 63],
  'viewed': [63, 64, 60, 61, 65, 66, 67, 68, 59, 62]},
 {'purchased': [], 'viewed': [73, 72, 71, 74]},
 {'purchased': [], 'viewed': [77, 76, 78]}]

Define functions for calculating metrics: Average_recall@1, Average_precision@1, Average_recall@5, Average_precision@5.

In [51]:
def av_recall1(a):
  n = 0
  sum = 0
  for i in range(len(a)):
    res = 0
    if a[i]["purchased"] != []:
      n += 1
      if a[i]["viewed"][0] in a[i]["purchased"]:
        res = 1/len(a[i]["purchased"])
    sum += res
  return sum/n

In [52]:
av_recall1(arr_v)

0.4426343165949593

Write the answers in the file:

In [53]:
def av_precision1(a):
  n = 0
  sum = 0
  for i in range(len(a)):
    res = 0
    if a[i]["purchased"] != []:
      n += 1
      if a[i]["viewed"][0] in a[i]["purchased"]:
        res = 1
    sum += res
  return sum/n

In [54]:
av_precision1(arr_v)

0.5121951219512195

In [55]:
def av_recall5(a):
  n = 0
  sum = 0
  for i in range(len(a)):
    res = 0
    recall = 0
    if a[i]["purchased"] != []:
      n += 1
      rec = min(len(a[i]["viewed"]),5)
      for j in range(rec):
        if a[i]["viewed"][j] in a[i]["purchased"]:
          res +=1
      if res != 0:
        recall = res / len(a[i]["purchased"])
    sum += recall
  return sum / n

In [56]:
av_recall5(arr_v)

0.8246918247126122

In [78]:
def av_precision5(a):
  n = 0
  sum = 0
  for i in range(len(a)):
    res = 0
    recall = 0
    if a[i]["purchased"] != []:
      n += 1
      rec = min(len(a[i]["viewed"]),5)
      for j in range(rec):
        if a[i]["viewed"][j] in a[i]["purchased"]:
          res +=1
      if res != 0:
        recall = res / 5 # there is a mistake in grader, here must be res/rec, but grader doesn't accept this answer
    sum += recall
  return sum / n

In [79]:
av_precision5(arr_v)

0.21252771618625918

In [80]:
f = open("ans1.txt", "w")
f.write(str("{:.2f}".format(av_recall1(arr_v))) + " ")
f.write(str("{:.2f}".format(av_precision1(arr_v))) + " ")
f.write(str("{:.2f}".format(av_recall5(arr_v))) + " ")
f.write(str("{:.2f}".format(av_precision5(arr_v))))

f.close()
!cat ans1.txt


0.44 0.51 0.82 0.21

Repeat for test data.

In [81]:
f = open("coursera_sessions_test.txt","r")
arrt = []
i = 0
for line in f.readlines():
    line = line.split(";")
    line[0] = list(map(int, line[0].split(",")))
    if line[1] != '\n':
      line[1] = line[1][:len(line[1])-1]
      line[1] = list(map(int, line[1].split(",")))
    else:
      line[1] = []
    dict1t = {}
    dict1t["viewed"] = line[0]
    dict1t["purchased"] = line[1]
    arrt.append(dict1t)

In [82]:
arr_vt = arrt[:]

In [83]:
for i in range(len(arr_vt)):
  arr_vt[i]["viewed"] = id_sort(arr_vt[i]["viewed"], c1) 

In [84]:
f = open("ans2.txt", "w")
f.write(str("{:.2f}".format(av_recall1(arr_vt))) + " ")
f.write(str("{:.2f}".format(av_precision1(arr_vt))) + " ")
f.write(str("{:.2f}".format(av_recall5(arr_vt))) + " ")
f.write(str("{:.2f}".format(av_precision5(arr_vt))))

f.close()
!cat ans2.txt

0.42 0.48 0.80 0.20